# Как искать в базе

* `lemma`. Введи лемму. Можно использовать регулярные выражения.

* `declencion_type`. Введи тип склонения. Можно использовать регулярные выражения.

* `gradation`. Введи тип лениции. Можно использовать регулярные выражения.

В следующих двух полях ты можешь задать существование какой-нибудь одной формы. Запрос к базе звучит так: выбери такие существительные, где форма сущ. {грамматические тэги} -- это {форма}.
Например: сущ. с тэгами NPOSS_NOM_SG = koira. Форму можно вводить, используя регулярные выражения, например, так: `tags = "NPOSS_NOM_SG"; form = ".*?ra"`

* `tags`. Грамматические тэги см. в gram_tags.txt. 

* `form`. Форма с заданным грамматическим значением. Можно использовать регулярные выражения (советую ограничиться . и .* -- остальное с большой вероятностью сломается)

##### Пожалуйста, не оставляй строки пустыми. Если ты не хочешь искать по этому параметру, введи `.*`

In [44]:
lemma = ".*" 
declencion_type = "10\*L.*" 
gradation = ".*" 

tags = "NPOSS_NOM_SG" 
form = "yl.*" 

## Пример. 
Запрос: 

`lemma = ".*"`

`declencion_type = "10\*L.*"` (да, не забудь экранировать те звездочки, которые нужно экранировать!)

`gradation = ".*"`

`tags = "NPOSS_NOM_SG"`

`form = "yl.*"`

Выдача: 

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>id</th>
      <th>word</th>
      <th>declencion_type</th>
      <th>gradation</th>
      <th>forms</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>16404</td>
      <td>ylkä</td>
      <td>10*L/koira</td>
      <td>k-j gradation</td>
      <td>{"NPOSS_class": "k-j gradation", "NPOSS_NOM_SG...</td>
    </tr>
  </tbody>
</table>
</div>


#### Не забудь запустить ячейку с полями. Если уже -- запускай ячейку ниже.

In [45]:
import sqlite3
import pandas as pd

like_form = form.replace('.*', '%').replace('.', '_')
path_tags = '$.'+tags

print('Поиск начался, жди :)')
conn = sqlite3.connect('FIN_nouns.db')
cursor = conn.cursor()

import sqlite3
import re

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

conn.create_function("REGEXP", 2, regexp)

q = """SELECT * FROM data 
WHERE word REGEXP ?
  AND declencion_type REGEXP ?
  AND gradation REGEXP ?
  AND JSON_EXTRACT(forms, ?) LIKE ?;"""

cursor.execute(q, (lemma, declencion_type, gradation, path_tags, like_form)) 
results = cursor.fetchall()

# Преобразование в DataFrame
df = pd.DataFrame(results, columns=[desc[0] for desc in cursor.description])

# Вывод DataFrame
print('Готово! Открой результат в ячейке ниже.')

conn.close()

Поиск начался, жди :)
Готово! Открой результат в ячейке ниже.


In [46]:
df

,id,word,declencion_type,gradation,forms
0,16404,ylkä,10*L/koira,k-j gradation,"{""NPOSS_class"": ""k-j gradation"", ""NPOSS_NOM_SG..."
